In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
!ls

Homework 6.ipynb Report.md        ccdefault.csv


In [3]:
data = pd.read_csv("ccdefault.csv")

In [4]:
data.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,DEFAULT
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [5]:
# Target is a little bit imbalanced
data["DEFAULT"].sum()/data.shape[0]

0.2212

# Random Test Train Splits

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [7]:
in_sample_scores = []
out_sample_scores = []

In [8]:
for i in range(1, 11):
    X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=["DEFAULT"]), data['DEFAULT'], random_state=i, stratify=data['DEFAULT'], test_size=0.1)
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_pred = model.predict(X_test)
    out_sample_scores.append(accuracy_score(y_test, y_pred))
    in_sample_scores.append(accuracy_score(y_train, y_train_pred))

In [9]:
# in_sample score
np.mean(in_sample_scores), np.var(in_sample_scores)

(1.0, 0.0)

In [10]:
# out_sample score
np.mean(out_sample_scores), np.var(out_sample_scores)

(0.7224999999999999, 6.376111111111133e-05)

# Cross Validation

In [11]:
from sklearn.model_selection import cross_validate

In [12]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=["DEFAULT"]), data['DEFAULT'], random_state=42, stratify=data['DEFAULT'], test_size=0.1)

In [13]:
# First use cross validation to fit and optimize model
model = DecisionTreeClassifier()
cv_result = cross_validate(model, X = X_train, y = y_train, cv=10)

In [14]:
cv_result["test_score"]

array([0.71666667, 0.72      , 0.72185185, 0.72222222, 0.72296296,
       0.72259259, 0.72518519, 0.72666667, 0.73037037, 0.72296296])

In [15]:
# in_sample score
np.mean(cv_result["test_score"]), np.var(cv_result["test_score"])

(0.7231481481481481, 1.235253772290806e-05)

In [16]:
# After cross-validation, we select the best model based on the average performance across all folds
# then retrain that model using the entire training dataset to test out-sample data

# since we don't make parameter tuning when conduct cross validation, this has no difference compare with a random split
model.fit(X_train, y_train)
y_pred = model.predict(X_test) 
accuracy_score(y_test, y_pred)

0.7273333333333334

# Conclusions 
Two methods are similar I think, even though in random test train splits, the in-sample accuracies are 100%. When we are tuning the model, we actually don't care and inspect out-sample's result to iterate our model's parameters. The difference is that cross validation is kind of random test train splits without replacement.<br>
I couldn't really tell which method provides the best estimate, because the accuracy on test data are actually similar.  If I have to make a choice, I think cross validation method generally performs better. <br>
For the efficiency, I think if they both run the same number of iterations, like in our quations both are 10 iterations, then there is no much difference. However, since cross validation is a split without replacement, it's more likely to require fewer iterations to let each part of the data serves as validation set. Thus, in general, random test train splits require more iteration to fully make sure every part of data have appeared in validation and takes more time.